In [ ]:
def aggregate_news_daily(news_df: ):
        """
        This function aggregates the news data by date and section name.
        It counts the number of articles published for each section on each date.
        :param news_df: DataFrame containing news data with columns 'pub_date', 'section_name', and 'news_desk'.

        :return: DataFrame with columns 'date', 'section_name', and 'num_mentioned' representing the count of articles.
        
        """
    
    news_df.createOrReplaceTempView("news_table")

    # query to retrieve the data
    sql_query = """
        WITH news_date_format as(
            SELECT 
                date(pub_date) as date,
                section_name,
                news_desk
            FROM news_table)

        SELECT 
            date,
            section_name,
            count(*) as num_mentioned
        FROM news_date_format
            GROUP BY date, section_name        
    """

    # query 
    aggr_df = spark.sql(sql_query)

    return aggr_df